In [9]:
import gymnasium as gym
import gymnasium_robotics
from gymnasium.wrappers import FlattenObservation

# Registrar y crear entorno
gym.register_envs(gymnasium_robotics)

#env = gym.make("FetchPickAndPlaceDense-v3")
env = gym.make("FetchPickAndPlaceDense-v3", max_episode_steps=150)

env = FlattenObservation(env)  # Combina dict en vector para SB3



In [13]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

# Wrap para usar con SB3
vec_env = DummyVecEnv([lambda: Monitor(env)])

# Crear modelo PPO
model = PPO("MlpPolicy", vec_env, verbose=1, tensorboard_log="./ppo_fetch_tensorboard/")

# Entrenar
model.learn(total_timesteps=100_000, reset_num_timesteps=False)
  # Podés aumentar si querés mejor resultado


Using cuda device
Logging to ./ppo_fetch_tensorboard/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 150      |
|    ep_rew_mean     | -91.5    |
|    success_rate    | 0        |
| time/              |          |
|    fps             | 415      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 150          |
|    ep_rew_mean          | -86.5        |
|    success_rate         | 0            |
| time/                   |              |
|    fps                  | 371          |
|    iterations           | 2            |
|    time_elapsed         | 11           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0063925623 |
|    clip_fraction        | 0.06         |
|    cli

In [14]:
model.save("ppo_fetch_pickandplace")

In [15]:
import numpy as np

def evaluar_agente(model, env, n_episodios=20):
    recompensas = []
    for _ in range(n_episodios):
        obs, _ = env.reset()
        total_reward = 0
        for _ in range(100):
            action, _ = model.predict(obs)
            obs, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            if terminated or truncated:
                break
        recompensas.append(total_reward)
    return recompensas

# Evaluar
rewards_eval = evaluar_agente(model, env, n_episodios=20)
print("Recompensas:", rewards_eval)
print("Promedio:", np.mean(rewards_eval))


Recompensas: [np.float64(-75.37648778452083), np.float64(-42.501880273377004), np.float64(-41.93785612208681), np.float64(-61.7506892047364), np.float64(-46.60967766975095), np.float64(-43.2793339242844), np.float64(-75.61546838688653), np.float64(-45.07722394870481), np.float64(-42.936356424394276), np.float64(-45.54354016049299), np.float64(-44.690620955533944), np.float64(-50.92060961101217), np.float64(-48.697831315207175), np.float64(-45.35375541798118), np.float64(-87.32006424730163), np.float64(-54.065706342414046), np.float64(-49.60213237631639), np.float64(-88.37657514789224), np.float64(-50.37151123781462), np.float64(-76.30730794943977)]
Promedio: -55.81673142500741
